## Sentiment/emotion analysis
จำแนกอารมณ์ของประโยค 

### Introduction : 
โดยทั่วไปแล้ว เราจะไม่สามารถกำหนดหรือปรับเปลี่ยน Model สำหรับการทำ Sentiment analysis ได้ 

แต่ ณ​ ปัจจุบันเราสามารถใช้ LLMs ให้การทำ Sentiment analysis จากตัวอย่างที่เราป้อนเข้าไปให้กับ Model โดยไม่จำเป็นต้อง Train Model

อาศัยเทคนิคที่เรียกว่า Few-shot prompting เพื่อให้ Model สามารถเรียนรู้ Pattern ก่อนหน้านี้ เพื่อจำแนกอารมณ์ของประโยคได้ 

### สิ่งที่แตกต่างจาก Traditional Sentiment Analysis :
- Flexible behaviour หรือการที่สามารถกำหนดบริบทของข้อมูลในการจำแนกอารมณ์จากประโยคได้
  
  แตกต่างจาก Traditional ที่ถูก Train ด้วย Data บางกลุ่มธุรกิจ และเมื่อนำมาใช้งานอาจจะไม่ได้แม่นยำ
- More output spectrume การที่สามารถกำหนดจำนวนของผลลัพธ์ตัวอย่างด้วยตัวเอง จะทำให้สามารถนำไปใช้งานต่อได้อย่างหลากหลายมากยิ่งขึ้น
  
  ไม่จำกัดแค่เพียง 3 ผลลัพธ์ ซึ่งเราสามารถกำหนดจำนวนหรือประเภทของผลลัพธ์ได้ด้วยตัวเอง

### Use case : 
- Transform data จาก Unstrcture data ให้กลายเป็น Structure data

### VS GPT-3.5 : 
ราคาต่อการประมวลผล สามารถทดสอบนำข้อมูลคำนวนผ่าน Tokenizer ได้ที่ https://float16.cloud/tokenizer โดยจะแสดงราคาต่อการประมวลผล และจำนวนของ Token ที่ใช้ในการประมวลผล

ทั่วไปแล้วราคาของ Float16 (SeaLLM-7b) จะถูกกว่า GPT-3.5 ถึง 90 - 95% หรือประมวลผลได้มากกว่า GPT-3.5 ถึง 10 เท่า ในราคาที่เท่ากัน

### Setup : 
<b>API Key :</b>

https://float16.cloud

<b>Dataset :</b>

https://huggingface.co/datasets/wisesight_sentiment

https://huggingface.co/datasets/wongnai_reviews

<b>หมายเหตุ :</b>

จำนวนของข้อมูล (example + label) ควรมีอย่างต่ำประเภทผลลัพธ์ละ 3 ตัวอย่าง 

เช่น อยากจำแนก 3 ประเภทผลลัพธ์ (Positive, Negative, Neutral) จำเป็นต้องใช้ตัวอย่างทั้งหมด 3x3 = 9 ตัวอย่าง 

ถ้าใช้จำนวนตัวอย่างน้อยเกินไปจะทำให้ Model ได้ความแม่นยำที่ต่ำลง

In [70]:
FLOAT16_API_KEY = 'float16-xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'
FLOAT16_CUSTOM_URL = 'https://api.float16.cloud/v1/chat/completions'

In [71]:
def init_system_task(prompt):
    output = []
    output.append({
        'role' : 'system',
        'content' : prompt
    })
    return output

def add_fewshot(data,example,label) : 
    output = []
    output = data
    output.append({
        'role' : 'user',
        'content' : example
    })
    output.append({
        'role' : 'assistant',
        'content' : label
    })
    return output

def add_question(data,question) : 
    output = []
    output = data
    output.append({
        'role' : 'user',
        'content' : question
    })
    return output


In [72]:
import requests
import json
import dotenv 
import os

prompt = ""
pair_data = [{
        'example' :'ประโยค : สวยเข้มทุกมุมมองครับ ส่วนเติ้ลธนพล หล่อเท่ห์มากครับ #ToyotaBuzz #เติ้ลธนพล',
        'label' : 'อารมณ์ : Neutral'
        },
        {
        'example' :'ประโยค : ฉลองวันเกิดลาซาด้าครบรอบ 6 ปี กับ อัพ ภูมิพัฒน์ !!! ขนแบรนด์ชั้นนำทั้ง Benefit, Vistra, Inisfree, Laneige และที่ขาดไม่ได้ Garnier นั้นเอง!!! . วันเดียว วันที่ 23 เม.ย. เวลา บ่าย 2 โมงตรง !!! อย่าลืมมาดูกันน้าาา ช้อป Lazada 25-27 เม.ย. ลดสูงสุด 90% >>> https://lzd.co/2qV2LGy',
        'label' : 'อารมณ์ : Neutral'
        },{
        'example' :'ประโยค : สวยจังแต่ข่อยใช้รีโว่นะก้อประหยัดค่าน้ำมันดี',
        'label' : 'อารมณ์ : Positive'
        },{
        'example' :'ประโยค : ฝนหยุดตกเเว้ว วันนี้มาชิลกับน้องๆสาวเบียร์ช้าง ตลาดหลบมุม Vintage night Barzaarขุขันธ์ ดึกๆค่อยม่วนกันต่อด้านในโซนโรงเบียร์จ้ะ',
        'label' : 'อารมณ์ : Positive'
        },{
        'example' :'ประโยค : ประชาชาติ ฟันธง ศก.ไม่ดี - จีดีพีเทียม #NationTV',
        'label' : 'อารมณ์ : Negative'
        },
        {
        'example' :'ประโยค : อย่าเสือก เด๋วเหงือกสั่น ซีวิกคันแดง ขับอย่างนี้ 5555555555',
        'label' : 'อารมณ์ : Negative'
        },{
        'example' :'ประโยค : ขนลุกมากตอนนี้ ยึกยือมาก บาบีก้อนไม่ปลอดภัยอีกต่อไป อ่านละคิดภาพตามอะ ควรกินแต่อาหารสุกๆท่องไว้ๆ',
        'label' : 'อารมณ์ : Negative'
        },
        {
        'example' :'ประโยค : โอ้ยแงน่ารักกกกก💕 อี้ชิงจะจุ้บหัวน้อยๆของมินซอกง่ะ55555555 ตอนแรกเหมือนจะแกล้งๆเป่าๆ มินซอกหันมาทำหน้าเหว๋อเลย เกิดอะไรขึ้นอ้ะมะกี้ สักพักจุ้บจริง ได้! จุ้บมาจุ้ยกลับ ไม่โกง!! แง น่ารักกกกกก มุมิมาก ทั้งคู่เลย😹💘✨',
        'label' : 'อารมณ์ : Positive'
        }
    ]
def do_request(url, data):
    headers = {'content-type': 'application/json', 'Authorization': 'Bearer '+FLOAT16_API_KEY}
    response = requests.post(url, headers=headers, data=json.dumps(data))
    response_text = response.json()
    return response_text

message = init_system_task('Your are sentiment analysis system for Thai language.')
for i in pair_data:
    message = add_fewshot(message, i['example'],i['label'])
message = add_question(message,'ประโยค : เตือนภัย Grab อย่าผูกบัตรเครดิต เพราะเรียกรถแล้วโดนตัดบัตรทันที แต่ถ้ารถไม่มา ต้องทำเรื่องคืนเงิน/ปฏิเสธรายการเอง ผลักภาระให้ผู้บริโภค ไร้ความรับผิดชอบที่สุด')
data = {
"messages" : message,
"model" : 'openthaigpt-70b',
"max_tokens" : 64,
"temperature" : 1
}
res = do_request(FLOAT16_CUSTOM_URL, data)
print(res['choices'][0]['message']['content'])

อารมณ์ : Negative
